# README

1. Install pytorch and detectron.
2. After detectron2 installed, the runtime needs to be restarted. Thus you can see a code exit(0) to restart the runtime and an error message: "你的工作階段因不明原因而異常終止。" shown. This is a correct result. There is no problem to go on executing the program.
3. The program will download the dataset, including `train_images.zip`, `pascal_train.json`, `test_images.zip` and `test.json`. And `train_images.zip` and `test_images.zip` will be automatically unzipped into `./train_images` and `./test_images`.
4. You can see the length of train dataset is 1349 and the list of 20 categories by checking coco.cats.
5. Two functions `datasetCateId_2_modelCateId` and `modelCateId_2_datasetCateId` are used to transfer between dataset category id and model category id. It is because dataset category id is 1 ~ 20 and model category id should be 0 ~ 19.
6. In detectron2, the dataset should be registered, such as:
<pre><code>DatasetCatalog.register('tiny_voc_train', lambda: get_tiny_voc_dicts('./train_images', './pascal_train.json', 'train'))
MetadataCatalog.get('tiny_voc_train').thing_classes = tiny_voc_classes('./train_images', './pascal_train.json') </code></pre>
7. In `get_tiny_voc_dicts`, we split the 1349 samples in train dataset into train dataset (1079 samples) and validate dataset (270 samples) and specify by the third parameter 'train' or 'valid'.
8. In Inference & evaluation using the trained model section. I have uploaded 7 models I trained in my top mAP list: mAP 64.238% in epoch 48999, mAP 64.129% in epoch 41999, mAP 63.109% in epoch 33999, mAP 62.736% in epoch 61999, mAP 61.604% in epoch 56999, mAP 61.367% in epoch 52999 and mAP 60.899% in epoch 63999.
9. In the colab, I use mAP 64.238% in epoch 48999, `model_0048999.pth`, as a demo model.
10. In Prepare submission file section, each instance is recorded in the list of `image_id`, `score`, `category_id` and `segmentation`. The instance mask map is converted by the function `binary_mask_to_rle` and is put into the variable `segmentation`.
11. The final submission file can be obtained from the file of `./output/submission.json`.
12. In Train section, we will resume the train work from epoch 63999.
13. In train phase, data augmentation of `RandomFlip`, `RandomBrightness`, `RandomContrast`, `RandomLighting` and `RandomRotation` are used.
14. The 1349 samples of train dataset are used to validate the mAP and submission jason format as well.

# Import Packages

In [ ]:
complete_submission = True

In [ ]:
if not complete_submission:
    !/opt/conda/bin/python3.7 -m pip install --upgrade pip

In [ ]:
if not complete_submission:
    # !pip install -q -U torch torchvision -f https://download.pytorch.org/whl/torch_stable.html
    !pip install torch===1.7.1 torchvision===0.8.2 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
if not complete_submission:
    !pip install -q -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
if not complete_submission:
    !pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/index.html

In [ ]:
if not complete_submission:
    !pip install googledrivedownloader

In [ ]:
if not complete_submission:
    !pip install bbox-visualizer

In [ ]:
# import common utilities
from contextlib import contextmanager
import cv2
import glob
from itertools import groupby
import json
import matplotlib.pyplot as plt
import random
import numpy as np
import os
import pandas
import re
import sys

# import coco python api
from pycocotools import mask as maskutil
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from google_drive_downloader import GoogleDriveDownloader as gdd

# import pytorch utilities
import torch
import torchvision
assert torch.__version__.startswith("1.7")

# import bbox visualizer
import bbox_visualizer as bbv

# Install detectron2

In [ ]:
# import detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, build_detection_train_loader, build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
import detectron2.data.transforms as T
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer

In [ ]:
# Setup detectron2 logger
setup_logger()

# Prepare Kaggle Wheat Competition Dataset

In [ ]:
!ls

## Show Dataset Information

In [ ]:
i = glob.glob('/kaggle/input/global-wheat-detection/train/*.jpg')
print('number of train images: {}'.format(len(i)))
print('train images: {}'.format(i))

i = glob.glob('/kaggle/input/global-wheat-detection/test/*.jpg')
print('number of test images: {}'.format(len(i)))
print('test images: {}'.format(i))

## Preprocess Kaggle Wheat Competition Dataset

In [ ]:
df = pandas.read_csv('/kaggle/input/global-wheat-detection/train.csv')
print(df)

In [ ]:
for i in df:
    print(i)

In [ ]:
wheat_list = df.to_dict('records')
print('number of label records: {}'.format(len(wheat_list)))

In [ ]:
for wheat in wheat_list[0:10]:
    print(wheat)

In [ ]:
images = [wheat['image_id'] for wheat in wheat_list]
images = list(dict.fromkeys(images))
print('number of wheat images: {}'.format(len(images)))

image_name2id = {}
for i, img in enumerate(images):
    # print(i, img)
    image_name2id[img] = i + 1
print('Image Name to ID mapping: {}'.format(image_name2id))

image_id2name = {}
for i, img in enumerate(images):
    # print(i, img)
    image_id2name[i+1] = img
print('Classes ID to Name mapping: {}'.format(image_id2name))

In [ ]:
wheat_classes = [wheat['source'] for wheat in wheat_list]
wheat_classes = list(dict.fromkeys(wheat_classes))
wheat_classes.sort()
print('wheat classes: {}'.format(wheat_classes))

num_classes = len(wheat_classes)
print('num of classes: {}'.format(num_classes))

class_name2id = {}
for i, c in enumerate(wheat_classes):
    # print(i, c)
    class_name2id[c] = i + 1
print('Classes Name to ID mapping: {}'.format(class_name2id))

class_id2name = {}
for i, c in enumerate(wheat_classes):
    # print(i, c)
    class_id2name[i+1] = c
print('Classes ID to Name mapping: {}'.format(class_id2name))

In [ ]:
wheat_dicts = {}
for idx, wheat in enumerate(wheat_list):
    if not wheat['image_id'] in wheat_dicts:
        image_id = wheat['image_id']
        wheat_dicts[image_id] = {}
        wheat_dicts[image_id]['file_name'] = os.path.join('./train', wheat['image_id']+'.jpg')
        wheat_dicts[image_id]['image_id'] = image_name2id[wheat['image_id']]
        wheat_dicts[image_id]['height'] = wheat['height']
        wheat_dicts[image_id]['width'] = wheat['width']
        wheat_dicts[image_id]['annotations'] = []
        ann = {}
        regex = re.compile(r'\[(\d*\.?\d*), (\d*\.?\d*), (\d*\.?\d*), (\d*\.?\d*)\]')
        match = regex.search(wheat['bbox'])
        ann['bbox'] = [float(match.group(1)), float(match.group(2)), float(match.group(3)), float(match.group(4))]
        ann['bbox_mode'] = BoxMode.XYWH_ABS
        ann['segmentation'] = []
        ann['category_id'] = class_name2id[wheat['source']]
        ann['id'] = idx + 1
        wheat_dicts[image_id]['annotations'].append(ann)
    else:
        ann = {}
        regex = re.compile(r'\[(\d*\.?\d*), (\d*\.?\d*), (\d*\.?\d*), (\d*\.?\d*)\]')
        match = regex.search(wheat['bbox'])
        ann['bbox'] = [float(match.group(1)), float(match.group(2)), float(match.group(3)), float(match.group(4))]
        ann['bbox_mode'] = BoxMode.XYWH_ABS
        ann['segmentation'] = []
        ann['category_id'] = class_name2id[wheat['source']]
        ann['id'] = idx + 1
        wheat_dicts[image_id]['annotations'].append(ann)

print(len(wheat_dicts))

In [ ]:
for wheat in wheat_dicts:
    print(wheat_dicts[wheat])

In [ ]:
temp_dir = './temp'
os.makedirs(temp_dir, exist_ok=True)

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

def get_wheat_dicts(wheat_dicts, mode='train'):

    dataset_dicts = []

    train_valid_split = 5
    for idx, wheat in enumerate(wheat_dicts):
        if mode == 'train' and (idx % train_valid_split) == 0:
            continue
        if mode == 'valid' and not (idx % train_valid_split) == 0:
            continue

        dataset_dicts.append(wheat_dicts[wheat])

    print('mode: {}, number of samples: {}'.format(mode, len(dataset_dicts)))
    return dataset_dicts

In [ ]:
train_dataset_dicts = get_wheat_dicts(wheat_dicts, 'train')
print(train_dataset_dicts[0:10])

train_dataset_coco_dicts = {}
train_dataset_coco_dicts['annotations'] = [{'segmentation':[
                        [ann['bbox'][0], ann['bbox'][1],
                        ann['bbox'][0]+ann['bbox'][2], ann['bbox'][1],
                        ann['bbox'][0]+ann['bbox'][2], ann['bbox'][1]+ann['bbox'][3],
                        ann['bbox'][0], ann['bbox'][1]+ann['bbox'][3]]
           ],
           'area':ann['bbox'][2]*ann['bbox'][3],
           'iscrowd':0,
           'image_id':img['image_id'],
           'bbox':ann['bbox'],
           'category_id':ann['category_id'],
           'id': ann['id']}
           for img in train_dataset_dicts
           for ann in img['annotations']]
train_dataset_coco_dicts['images'] = [{'file_name':img['file_name'], 'id':img['image_id'], 'height':img['height'], 'width':img['width']} for img in train_dataset_dicts]
train_dataset_coco_dicts['categories'] = [{'supercategory':c, 'name':c, 'id':class_name2id[c]} for c in wheat_classes]

with open(os.path.join(temp_dir, 'wheat_train_coco.json'), 'w') as outfile:
    json.dump(train_dataset_coco_dicts, outfile)
outfile.close()

In [ ]:
valid_dataset_dicts = get_wheat_dicts(wheat_dicts, 'valid')
print('number of valid samples: {}', len(valid_dataset_dicts))
print(valid_dataset_dicts[0:10])

valid_dataset_coco_dicts = {}
valid_dataset_coco_dicts['annotations'] = [{'segmentation':[
                        [ann['bbox'][0], ann['bbox'][1],
                        ann['bbox'][0]+ann['bbox'][2], ann['bbox'][1],
                        ann['bbox'][0]+ann['bbox'][2], ann['bbox'][1]+ann['bbox'][3],
                        ann['bbox'][0], ann['bbox'][1]+ann['bbox'][3]]
           ],
           'area':ann['bbox'][2]*ann['bbox'][3],
           'iscrowd':0,
           'image_id':img['image_id'],
           'bbox':ann['bbox'],
           'category_id':ann['category_id'],
           'id': ann['id']}
           for img in valid_dataset_dicts
           for ann in img['annotations']]
valid_dataset_coco_dicts['images'] = [{'file_name':img['file_name'], 'id':img['image_id'], 'height':img['height'], 'width':img['width']} for img in valid_dataset_dicts]
valid_dataset_coco_dicts['categories'] = [{'supercategory':c, 'name':c, 'id':class_name2id[c]} for c in wheat_classes]

with open(os.path.join(temp_dir, 'wheat_valid_coco.json'), 'w') as outfile:
    json.dump(valid_dataset_coco_dicts, outfile)
outfile.close()

In [ ]:
!ls

In [ ]:
DatasetCatalog.clear()
from detectron2.data.datasets import register_coco_instances

register_coco_instances('wheat_train', {}, 'wheat_train_coco.json', temp_dir)
register_coco_instances('wheat_valid', {}, 'wheat_valid_coco.json', temp_dir)

In [ ]:
!rm ./temp/wheat_train_coco.json
!rm ./temp/wheat_valid_coco.json

In [ ]:
cfg = get_cfg()
# cfg.OUTPUT_DIR = '/content/drive/MyDrive/NCTU/基於深度學習之視覺辨識專論/HW/Final/output1'
cfg.MODEL.DEVICE = 'cpu'
cfg.OUTPUT_DIR = temp_dir
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ('wheat_train',)
cfg.DATASETS.TEST = ('wheat_valid',)
cfg.TEST.EVAL_PERIOD = 2000
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0000025  # 0.00025: 0~12999; 0.0001: 13000~
cfg.SOLVER.MAX_ITER = 300000  # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.GAMMA = 0.5
# cfg.SOLVER.STEPS = (60000,100000,210000,250000) 0.5 gamma
cfg.SOLVER.STEPS = (30000,)
# cfg.SOLVER.LR_SCHEDULER_NAME = WarmupMultiStepLR
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 0
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

In [ ]:
print(cfg)

# Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:

if not complete_submission:
    gdd.download_file_from_google_drive(file_id='12T8tCt1n8uplDLjeMypkbqWNofLu0Ine',
            dest_path=os.path.join(cfg.OUTPUT_DIR, 'model_0299999.pth'),
            unzip=False)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_0299999.pth')  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold

predictor = DefaultPredictor(cfg)

## Prepare submission file

In [ ]:
import pandas as pd
import numpy as np

testfiles = [f for f in os.listdir('/kaggle/input/global-wheat-detection/test') if os.path.isfile(os.path.join('/kaggle/input/global-wheat-detection/test', f))]
# print(testfiles)

filenames = []
predicts = []

for f in testfiles:
    # print('image filename: {}'.format(f))
    # print(os.path.join('/kaggle/input/global-wheat-detection/test', f))
    image = cv2.imread(os.path.join('/kaggle/input/global-wheat-detection/test', f))

    outputs = predictor(image)

    result = {}

    # result['image_id'] = f['id']
    result['image_id'] = os.path.splitext(f)[0]
    result['score'] = [[round(i.tolist(),2)] for i in outputs['instances'].scores]
    result['category_id'] = [cate_id.numpy()+1 for cate_id in outputs["instances"].pred_classes.to('cpu')]
    result['bbox'] = [[round(box.tolist()[0]),
                   round(box.tolist()[1]),
                   round(box.tolist()[2]-box.tolist()[0]),
                   round(box.tolist()[3]-box.tolist()[1])] for box in outputs["instances"].pred_boxes]

    pred = []
    n_instances = len(outputs['instances'].scores)
    # print(n_instances)
    for i in range(n_instances):  # Loop all instances
        # save information of the instance in a dictionary then append on coco_dt list
        pred.extend(result['score'][i])
        pred.extend(result['bbox'][i])

    filenames.append(result['image_id'])
    predicts.append(pred)
    # print(pred)

print('filename list: {}'.format(filenames))
# print('predict list: {}'.format(predicts))
predicts = [' '.join(str(p) for p in pred) for pred in predicts]
print('predict list: {}'.format(predicts))

submission_dict = {
    'image_id': filenames,
    'PredictionString': predicts
}
submission = pd.DataFrame(submission_dict)
# submission.to_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'), encoding = 'utf-8',index = False)

# submission.to_json(os.path.join('./', 'submission.json'), index = False)

In [ ]:
submission.to_csv(os.path.join('./', 'submission.csv'), encoding = 'utf-8',index = False)